## 02. SQL

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

- SQL Database 로드

In [2]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

In [3]:
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

print(db.dialect)
print(db.get_usable_table_names())

sqlite
['accounts', 'customers', 'transactions']


<br>

- LLM 객체를 생성하고 LLM 과 DB 를 매개변수로 입력하여 `chain` 을 생성

In [5]:
from langchain_core.prompts import PromptTemplate

In [6]:
prompt = PromptTemplate.from_template(
    """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Here is the description of the columns in the tables:
`cust`: customer name
`prod`: product name
`trans`: transaction date

Question: {input}"""
).partial(dialect=db.dialect)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db, prompt)

- `chain` 을 실행하면 DB 기반으로 쿼리를 생성

In [7]:
generated_sql_query = chain.invoke({"question": "고객의 이름을 나열하세요"})

# 생성된 쿼리를 출력
print(generated_sql_query.__repr__())

'SELECT name\nFROM customers'


<br>

- 생성한 쿼리가 맞게 동작하는지 확인

In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [11]:
execute_query = QuerySQLDataBaseTool(db=db)

# SQL 쿼리 생성 체인
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query

In [12]:
chain.invoke({"question": "테디의 이메일을 조회하세요"})

"[('teddy@example.com',)]"

<br>

### 답변을 LLM으로 증강-생성
- LCEL의 문법의 체인으로 좀 더 자연스러운 답변을 받을 수 있도록 조정

In [13]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [14]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [15]:
answer = answer_prompt | llm | StrOutputParser()

In [16]:
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [17]:
chain.invoke({"question": "테디의 transaction 의 합계를 구하세요"})

'테디의 transaction의 합계는 -965.7 입니다.'

<br>

### Agent
- Agent를 활용하여 SQL 쿼리를 생성하고, 실행 결과를 답변으로 출력

In [18]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [20]:
agent_executor.invoke(
    {"input": "테디와 셜리의 transaction 의 합계를 구하고 비교하세요"}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(account_id) REFERENCES accounts (account_id)
)

/*
3 rows from transactions table:
transaction_id	account_id	amount	transaction_date
1	1	74.79	2024-07-13
2	1	-224.1	2024-05-13
3	1	-128.9	2024-01-25
*/
Invoking: `sql_db_query` with `{'query': 'SELECT SUM(amount) AS total_amount FROM transactions WHERE account_id = 1'}`


[(-965.7,)]
Invoking: `sql_db_query` with `{'query': 'SELECT SUM(amount) AS total_amount FROM transactions WHERE account_id = 2'}`


[(743.13,)]테디의 거래 합계는 -965.7이고, 셜리의 거래 합계는 743.13입니다.

> Finished chain.


{'input': '테디와 셜리의 transaction 의 합계를 구하고 비교하세요',
 'output': '테디의 거래 합계는 -965.7이고, 셜리의 거래 합계는 743.13입니다.'}

<br>

<hr>

<br>

## 03. 구조화된 출력 체인(with_structered_output)

<br>

#### 특정 주제에 대한 4지선다형 퀴즈를 생성하는 과정을 구현

In [21]:
from langchain_classic.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

- `Quiz` 클래스는 퀴즈의 질문, 난이도, 그리고 네 개의 선택지를 정의

In [22]:
class Quiz(BaseModel):
    """4지선다형 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 4개의 선택지 입니다.")


- ChatOpenAI 인스턴스는 GPT-4o 모델을 사용하여 자연어 처리를 수행하고, `ChatPromptTemplate`는 퀴즈 생성을 위한 대화형 프롬프트를 정의

In [23]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 4지선다형 퀴즈를 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

- 구조화된 출력을 위한 모델 생성

In [24]:
llm_with_structured_output = llm.with_structured_output(Quiz)

- 퀴즈 생성 체인 생성

In [25]:
chain = prompt | llm_with_structured_output

- 퀴즈 생성을 요청

In [26]:
generated_quiz = chain.invoke({"topic": "ADSP(데이터 분석 준전문가) 자격 시험"})

In [27]:
print(f"{generated_quiz.question} (난이도: {generated_quiz.level})\n")
for i, opt in enumerate(generated_quiz.options):
    print(f"{i+1}) {opt}")

데이터 분석에서 '회귀 분석'이란 무엇을 예측하기 위한 기법인가요? (난이도: 보통)

1) 연속형 변수
2) 범주형 변수
3) 이산형 변수
4) 순서형 변수


<br>

<hr>